# Supervised Classification

In [1]:
import ee
ee.Initialize()
ee.Authenticate()

True

In [2]:
import geemap

rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
palette = ['#cc6d8f', '#ffc107', '#1e88e5', '#004d40']

In [3]:
banglore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
geometry = banglore.geometry()

In [4]:
urban = ee.FeatureCollection('users/ujavalgandhi/e2e/urban_gcps')
bare = ee.FeatureCollection('users/ujavalgandhi/e2e/bare_gcps')
water = ee.FeatureCollection('users/ujavalgandhi/e2e/water_gcps')
vegetation = ee.FeatureCollection('users/ujavalgandhi/e2e/vegetation_gcps')

In [5]:
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
s2 = (
    s2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))
    .filter(ee.Filter.bounds(geometry))
    .select('B.*')
)
s2_composite = s2.median()

ee.Image.sampleRegions

Converts each pixel of an image (at a given scale) that intersects one or more regions to a Feature, returning them as a FeatureCollection. Each output feature will have one property per band of the input image, as well as any specified properties copied from the input feature.
Note that geometries will be snapped to pixel centers.

In [6]:
gcps = urban.merge(bare).merge(water).merge(vegetation)
training = s2_composite.sampleRegions(
    collection=gcps,
    properties=['landcover'],
    scale=10,
)
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='landcover',
    inputProperties=s2_composite.bandNames()
)
classified = s2_composite.classify(classifier)

In [7]:
Map = geemap.Map()
Map.addLayer(geometry, {}, 'Banglore')
Map.addLayer(urban.geometry(), {'color':'red'}, 'Urban')
Map.addLayer(bare.geometry(), {'color':'brown'}, 'Bare')
Map.addLayer(water.geometry(), {'color':'blue'}, 'Water')
Map.addLayer(vegetation.geometry(), {'color':'green'}, 'Vegetation')
Map.addLayer(s2_composite.clip(geometry), rgb_vis, "Sentinel-2")
Map.addLayer(classified.clip(geometry), {'min': 0, 'max': 3, 'palette': palette}, 'Landcover')
Map.centerObject(geometry)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Train with train val split

In [ ]:
basin = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_7')
gcp = ee.FeatureCollection('users/ujavalgandhi/e2e/arkavathy_gcps')

arkavathy = basin.filter(ee.Filter.eq('HYBAS_ID', 4071139640))
arkavathy_geometry = arkavathy.geometry()

# split train validation
gcp = gcp.randomColumn('random')
training_gcp = gcp.filter(ee.Filter.lt('random', 0.6))
validation_gcp = gcp.filter(ee.Filter.gte('random', 0.6))

training = s2_composite.sampleRegions(
    collection=training_gcp,
    properties=['landcover'],
    scale=10,
    tileScale=16,
)

classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='landcover',
    inputProperties=s2_composite.bandNames()
)

classified = s2_composite.classify(classifier)

In [ ]:
validation = s2_composite.sampleRegions(
    collection=validation_gcp,
    properties=['landcover'],
    scale=10,
    tileScale=16,
)
test = validation.classify(classifier)

confusion_matrix = test.errorMatrix('landcover', 'classification')
print(confusion_matrix.getInfo())
print('Validation overall accuracy: ', confusion_matrix.accuracy().getInfo())

# Improved Classification

In [ ]:
def mask_cloud_shadow_sr(image: ee.Image) -> ee.Image:
    cloud_prob = image.select('MSK_CLDPRB')
    cloud = cloud_prob.lt(10)
    scl = image.select('SCL')
    shadow = scl.eq(3) # cloud shadow
    cirrus = scl.eq(10) # cirrus
    mask = cloud.And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask)

def add_incidies(image: ee.Image) -> ee.Image:
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    mndwi = image.normalizedDifference(['B3', 'B8']).rename('mndwi')
    bsi = image.expression(
        '((X + Y) - (A - B)) / ((X + Y) + (A + B))', {
            'X': image.select('B11'), # swir1
            'Y': image.select('B4'), # red
            'A': image.select('B8'), # nir
            'B': image.select('B2') # blue
        }
    )
    return image.addBands(ndvi).addBands(ndbi).addBands(mndwi).addBands(bsi)

def normalize_bands(image: ee.Image, geometry: ee.Geometry) -> ee.Image:
    min_dict = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=geometry,
        scale=20,
        maxPixels=1e13,
        bestEffort=True,
        tileScale=16,
    )
    max_dict = image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=geometry,
        scale=20,
        maxPixels=1e13,
        bestEffort=True,
        tileScale=16,
    )
    mins = ee.Image.constant(min_dict.values(image.bandNames()))
    maxs = ee.Image.constant(max_dict.values(image.bandNames()))
    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized

s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
basin = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_7')
gcp = ee.FeatureCollection('users/ujavalgandhi/e2e/arkavathy_gcps')
alos = ee.Image('JAXA/ALOS/AW3D30/V2_2')

arkavathy = basin.filter(ee.Filter.eq('HYBAS_ID', 4071139640))

s2 = (
    s2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))
    .filter(ee.Filter.bounds(arkavathy.geometry()))
    .map(mask_cloud_shadow_sr)
    # .select('B.*')
)
s2_composite = s2.median()
composite = add_incidies(s2_composite)

elev = alos.select('AVE_DSM').rename('elev')
slope = ee.Terrain.slope(alos.select('AVE_DSM')).rename('slope')
composite = composite.addBands(elev).addBands(slope)
composite = normalize_bands(composite, arkavathy.geometry())

In [ ]:
gcp = gcp.randomColumn('random')

training_gcp = gcp.filter(ee.Filter.lt('random', 0.6))
validation_gcp = gcp.filter(ee.Filter.gte('random', 0.6))

training = composite.sampleRegions(
    collection=training_gcp,
    properties=['landcover'],
    scale=10,
    tileScale=16,
)

classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='landcover',
    inputProperties=composite.bandNames()
)

classified = composite.classify(classifier)

test = classified.sampleRegions(
    collection=validation_gcp,
    properties=['landcover'],
    scale=10,
    tileScale=16,
)

confusion_matrix = test.errorMatrix('landcover', 'classification')

In [ ]:
print(confusion_matrix.getInfo())
print('Validation overall accuracy: ', confusion_matrix.accuracy().getInfo())